In [6]:
import requests
import mysql.connector as dbconnect

def fetch_posts():
    url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

def load_to_mysql(data):
    try:
        conn = dbconnect.connect(
            host='localhost',
            port=3306,
            user='root',
            password='password',
            database='creditcard_capstone'
        )

        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS CDW_SAPP_loan_application (
                Application_ID VARCHAR(20) PRIMARY KEY,
                Gender VARCHAR(10),
                Married VARCHAR(10),
                Dependents VARCHAR(10),
                Education VARCHAR(50),
                Self_Employed VARCHAR(10),
                Credit_History INT,
                Property_Area VARCHAR(20),
                Income VARCHAR(20),
                Application_Status VARCHAR(5)
            )
        ''')

        expected_keys = [
            'Application_ID', 'Gender', 'Married', 'Dependents',
            'Education', 'Self_Employed', 'Credit_History',
            'Property_Area', 'Income', 'Application_Status'
        ]

        for info in data:
            try:
                if all(key in info for key in expected_keys):
                    cursor.execute('''
                        INSERT INTO CDW_SAPP_loan_application (
                            Application_ID, Gender, Married, Dependents, Education, 
                            Self_Employed, Credit_History, Property_Area, Income, Application_Status)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE
                            Gender=VALUES(Gender),
                            Married=VALUES(Married),
                            Dependents=VALUES(Dependents),
                            Education=VALUES(Education),
                            Self_Employed=VALUES(Self_Employed),
                            Credit_History=VALUES(Credit_History),
                            Property_Area=VALUES(Property_Area),
                            Income=VALUES(Income),
                            Application_Status=VALUES(Application_Status)
                    ''', (
                        info['Application_ID'],
                        info['Gender'],
                        info['Married'],
                        info['Dependents'],
                        info['Education'],
                        info['Self_Employed'],
                        info['Credit_History'],
                        info['Property_Area'],
                        info['Income'],
                        info['Application_Status']
                    ))
                else:
                    print(f"Skipping record due to missing keys: {info.get('Application_ID', 'UNKNOWN')}")
            except Exception as record_error:
                print(f"Error inserting record {info.get('Application_ID', 'UNKNOWN')}: {record_error}")

        conn.commit()
        cursor.close()
        conn.close()

        print("Data loaded successfully to MySQL database.")

    except Exception as e:
        print(f"Connection or setup error: {e}")

# Run it
data = fetch_posts()
if data:
    load_to_mysql(data)


Data loaded successfully to MySQL database.
